In [46]:
import pandas as pd
from preprocess import *
import random
random.seed(22)

In [47]:
# Import the users levels dataset which has been cleaned up
user_levels_df = pd.read_pickle('Data/users_levels_clean')

#Import the dataset containing the comments AFTER language detection
df = pd.read_pickle('Data/Classified/english_comments_2')

In [48]:
# merge with user levels to then separate native from non native speakers
df = df.merge(user_levels_df[['author', 'english']], left_on = 'author', right_on = 'author')

# separate native from non native speakers
natives_bool = (df['english'] == 'N')
natives = df[natives_bool]
non_natives = df[~natives_bool]

In [49]:
# Generate feeds for native and non native speakers separately 
native_feeds = generate_feeds(natives, nb_feeds = 20, nb_words_per_feed = 500, exact = False, seed = 0)
non_native_feeds = generate_feeds(non_natives, nb_feeds = 20, nb_words_per_feed = 500, exact = False, seed = 0)

print('The new datasets contains ',len(native_feeds), 'native and', len(non_native_feeds), 'non-native authors.')


100%|██████████| 325/325 [00:00<00:00, 780.07it/s]

The new datasets contains  354 native and 135 non-native authors.


In [50]:
# Save the newly generated dataframe into pickle file
native_feeds.to_pickle('Data/Feeds/native_english_20feeds')
non_native_feeds.to_pickle('Data/Feeds/non_native_english_20feeds')

In [51]:
# Create 2 dataframes for the tunning part 
# with 30 randomly picked natives for the first dataframe and 30 randomly picked non-natives for the other one.
tunning_samples_native     = random.sample(range(len(native_feeds)), 30)
tunning_samples_non_native = random.sample(range(len(non_native_feeds)), 30)

native_authors_tunning = native_feeds.iloc[tunning_samples_native]
non_native_authors_tunning = non_native_feeds.iloc[tunning_samples_non_native]

In [52]:
# Create 2 dataframes for the testing part
#  with 100 randomly picked natives for the first dataframe and 100 randomly picked non-natives for the other one.

# Drop the authors who have already been taken in the train set
test_native_feeds     = native_feeds.drop(index = native_authors_tunning.index)
test_non_native_feeds = non_native_feeds.drop(index = non_native_authors_tunning.index)

# Chose randomly 100 native authors and 100 non-native authors among the remaining
test_samples_native     = random.sample(range(len(test_native_feeds)), 100)
test_samples_non_native = random.sample(range(len(test_non_native_feeds)), 100)

native_authors_testing = test_native_feeds.iloc[test_samples_native]
non_native_authors_testing = test_non_native_feeds.iloc[test_samples_non_native]

In [57]:
# Save these new dataframes into a csv file
native_authors_tunning.to_csv('dataset/Tunning/30native_english.csv', sep = '\t')
non_native_authors_tunning.to_csv('dataset/Tunning/30non_native_english.csv', sep = '\t')
native_authors_testing.to_csv('dataset/Test/100native_english.csv', sep = '\t')
non_native_authors_testing.to_csv('dataset/Test/100non_native_english.csv', sep = '\t')

In [58]:
# Save these new dataframes into a pickle file
native_authors_tunning.to_pickle('dataset/Tunning/30native_english')
non_native_authors_tunning.to_pickle('dataset/Tunning/30non_native_english')
native_authors_testing.to_pickle('dataset/Test/100native_english')
non_native_authors_testing.to_pickle('dataset/Test/100non_native_english')

: 

In [59]:
pd.read_csv('dataset/Tunning/30native_english.csv', sep = '\t', index_col=0)

,timerange,file1,slices1,file2,slices2,file3,slices3,file4,slices4,file5,...,file16,slices16,file17,slices17,file18,slices18,file19,slices19,file20,slices20
author,,,,,,,,,,,,,,,,,,,,,
Itdoesntmatteronebit,150,"['Glad', 'it', 'was', 'useful.', 'B2', 'is', '...","[48, 24, 97, 23, 51, 52, 64, 141]","['Just', 'looked', 'like', 'a', 'hard', 'colla...","[64, 6, 71, 56, 17, 98, 27, 63, 41, 116]","[""It's"", 'like', 'this', 'guy', ""didn't"", 'loo...","[12, 106, 16, 231, 44, 74, 23]","['Same', 'thing.', 'Will', 'take', 'a', 'look'...","[2, 53, 21, 49, 7, 31, 28, 2, 10, 8, 57, 49, 4...","['American', 'here,', 'living', 'in', 'France'...",...,"['>', 'Rates', 'are', '7.5-8%', 'What', 'the',...","[24, 39, 5, 85, 62, 60, 29, 29, 18, 10, 8, 32,...","['Has', 'Jimenez', 'been', 'active', 'lately?'...","[17, 59, 7, 16, 6, 131, 1, 25, 8, 129, 48, 11,...","['""Will', 'I', 'get', 'better', 'at', 'X', 'if...","[70, 49, 9, 41, 27, 21, 107, 34, 9, 91, 41, 11]","['Good', 'points.', ""There's"", 'a', 'differenc...","[2, 114, 27, 111, 100, 17, 29, 11, 9, 69, 76]","['Is', 'it', 'asked', 'as', 'a', 'question', '...","[43, 42, 25, 53, 25, 57, 7, 30, 59, 34, 27, 8,..."
PM_MAJESTIC_PICS,98,"['What', 'is', 'a', '“1', 'year', 'MA', 'with'...","[33, 7, 16, 23, 5, 22, 8, 19, 38, 20, 38, 7, 4...","['Case', 'in', 'point:', 'she', 'doesn’t', 'un...","[23, 42, 6, 30, 7, 6, 4, 6, 46, 29, 16, 10, 18...","['I', 'saw', 'one', 'of', 'my', 'students', 'w...","[18, 14, 9, 10, 54, 22, 3, 25, 8, 1, 25, 16, 1...","['This', 'is', 'gorgeous', '❤️', 'Is', '“fit',...","[4, 24, 31, 15, 8, 20, 17, 7, 15, 6, 22, 17, 2...","['I', 'kept', 'my', 'U.S.', 'phone', 'active.'...",...,"['Who', 'is', 'singing', 'this??', 'This', 'is...","[36, 13, 10, 7, 14, 48, 20, 61, 64, 121, 5, 7,...","['I', 'think', 'it’s', 'that', 'sinful', 'thin...","[35, 5, 113, 33, 45, 20, 19, 88, 3, 19, 43, 71...","['I', 'really', 'liked', 'the', 'Season', '4',...","[44, 6, 39, 11, 4, 48, 14, 63, 28, 8, 51, 57, ...","['I', 'love', 'her', 'grumpy', 'faces—', 'she'...","[40, 35, 20, 6, 12, 4, 12, 15, 38, 14, 30, 15,...","['To', 'be', 'honest,', 'most', 'of', 'the', '...","[29, 44, 23, 5, 12, 63, 4, 20, 136, 94, 16, 54]"
Apprehensive-Mind532,316,"['The', 'end', 'of', 'Ready', 'or', 'Not,', 'g...","[9, 56, 104, 13, 101, 311]","['How', 'long', 'do', 'you', 'spend', 'on', 'y...","[108, 9, 69, 20, 96, 51, 162]","['I', 'think', ""it's"", 'just', 'about', 'needi...","[153, 61, 112, 37, 87, 56]","['Sounds', 'like', ""you're"", 'not', 'feeling',...","[63, 19, 17, 13, 11, 91, 48, 23, 8, 111, 43, 126]","['Start', 'with', 'German', 'and', 'maybe', 'p...",...,"['Nothing', 'quite', 'the', 'same,', 'but', 'c...","[83, 45, 99, 14, 57, 39, 1, 172]","['Great', 'movies', 'to', 'make', 'you', 'susp...","[22, 132, 58, 40, 3, 35, 37, 48, 50, 6, 56, 12...","['ITA.', 'AlienS', 'might', 'be', 'the', 'bett...","[7, 104, 31, 19, 4, 54, 69, 30, 48, 29, 13, 149]","['So', 'for', 'Japanese', 'and', 'Spanish', 'c...","[169, 33, 73, 34, 58, 57, 21, 37, 39]","['All', 'right', 'then,', 'come', 'on', 'over'...","[13, 5, 146, 38, 138, 51, 50, 247]"
sam-lb,289,"['No', 'way', 'is', 'this', 'harder', 'than', ...","[7, 46, 88, 338, 21]","['Impressive!', 'Good', 'luck.', 'People', 'ne...","[3, 3, 121, 9, 25, 6, 19, 27, 5, 15, 1, 17, 21...","['oh', 'I', 'do', 'plot', 'in', 'python,', 'bu...","[26, 43, 11, 25, 75, 33, 22, 18, 17, 7, 8, 38,...","['ah', 'okay,', 'I', 'see.', 'I', 'was', 'wond...","[32, 6, 2, 37, 10, 17, 7, 9, 135, 14, 1, 2, 13...","['Shoddy', 'rushed', 'work', 'in', 'a', 'vital...",...,"['Meh,', 'I', 'guess,', 'but', 'regardless', '...","[43, 5, 19, 31, 5, 22, 34, 76, 24, 14, 16, 10,...","[""Don't"", 'tell', 'me', ""it's"", 'about', 'puni...","[57, 13, 71, 23, 23, 15, 118, 43, 7, 5, 17, 72...","['Jawbreaker', 'is', 'fun', 'my', 'dude', ""Wou...","[5, 26, 40, 11, 87, 55, 13, 7, 12, 145, 31, 59...","['Envelope', 'of', 'a', 'cardioid', '(almost)....","[12, 20, 14, 12, 70, 15, 52, 110, 18, 12, 3, 2...","['Please', 'do.', 'This', 'is', 'a', 'gold', 